이번 챕터에서는 그 중 정보 검색과 텍스트 마이닝 분야에서 주로 사용되는 카운트 기반의 텍스트 표현 방법인 DTM(Document Term Matrix)과 TF-IDF(Term Frequency-Inverse Document Frequency)에 대해서 다룹니다.

텍스트를 위와 같은 방식으로 수치화를 하고나면, 통계적인 접근 방법을 통해 여러 문서로 이루어진 텍스트 데이터가 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내거나, 문서의 핵심어 추출, 검색 엔진에서 검색 결과의 순위 결정, 문서들 간의 유사도를 구하는 등의 용도로 사용할 수 있습니다.

## 1. Bag of words(Bow)

단어의 등장 순서를 고려하지 않은 빈도수 기반의 단어 표현 방법
1. 각 단어의 고유한 정수 인덱스를 부여
2. 각 인덱스 위치에 단어 토큰의 등장 횟수를 기록한 백터를 만든다.

doc1 = 'John likes to watch movies. Mary likes movies too'  
Bow1 = {"Jonh" : 1, "likes" :2, "to" : 1, "watch" :1, "movies" : 2, "Mary" : 1, "too" : 1}

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.7 MB/s 
     |████████████████████████████████| 448 kB 41.0 MB/s 


In [ ]:
from konlpy.tag import Okt
import re

okt = Okt()

# 정규표현식을 통해 온점을 제거하는 정제 작업
token = re.sub("(\.)", "", "소비자는 주로 소비하는 상품을 기준으로 물가 상승률을 느낀다.")
# print(token)
token = okt.morphs(token) # okt 형태소 분석

In [ ]:
token

['소비자',
 '는',
 '주로',
 '소비',
 '하는',
 '상품',
 '을',
 '기준',
 '으로',
 '물가',
 '상',
 '승률',
 '을',
 '느낀다']

In [ ]:
word2index = {}
bow = []

for voca in token:
    if voca not in word2index.keys():
        # 토큰을 읽으면서, word2index에 없는 단어는 새로 추가하고
        # 이미 있는 단어는 넘긴다.
        word2index[voca] = len(word2index) # {}
        # bow 전체에 전부 기본값을 1을 넣어준다. 단어 갯수는 최소 1개 이상이기 때문에
        bow.insert(len(word2index)-1, 1)
    else:
        index = word2index.get(voca)
        # 재 등장하는 단어의 인덱스를 받아오기
        bow[index] = bow[index] + 1
        # 재 등장한 단어의 해당하는 인덱스의 위치에 1을 더해줌( 단어 갯수를 세는 것)

print(word2index) # 단어장

{'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가': 9, '상': 10, '승률': 11, '느낀다': 12}


In [ ]:
bow

[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1]

## Tensorflow의 keras Tokenizer를 활용한 Bow

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sentence = ["John likes to watch movies. \
            Mary likes movies too! \
            Mary also likes to watch football games."]

In [ ]:
def print_bow(sentence):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentence)

    bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow 저장

    print("Bag of words : ", bow)
    print("단어장(vocabulary)의 크기 : ", len(tokenizer.word_counts))

In [ ]:
print_bow(sentence)

Bag of words :  {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(vocabulary)의 크기 :  10


## scikit-learn의 CountVectorizer을 활용한 Bow

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
sentence = ["John likes to watch movies. \
            Mary likes movies too! \
            Mary also likes to watch football games."]

sentence = ["John likes to watch movies. \
            Mary likes movies too! \
            나는가방에들어갔다. 아버지가방에들어가신다.?"]

In [ ]:
vector = CountVectorizer()

In [ ]:
print("Bag of words : ", vector.fit_transform(sentence).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록
print("각 단어의 인덱스 :", vector.vocabulary_) # 각 단어으

Bag of words :  [[1 2 1 2 1 1 1 1 1]]
각 단어의 인덱스 : {'john': 0, 'likes': 1, 'to': 4, 'watch': 6, 'movies': 3, 'mary': 2, 'too': 5, '나는가방에들어갔다': 7, '아버지가방에들어가신다': 8}


## 4. 불용어를 제거한 BoW 만들기

### 사용자가 직접 정의한 불용어 사용

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text = ["Family is not an importent thing. it's everything"]

vect = CountVectorizer(stop_words = ["the", "a", "an", "is", "not"])

print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'importent': 2, 'thing': 4, 'it': 3, 'everything': 0}


## CountVectorizer에서 제공하는 자체 불용어 사용

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")

print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


NLTK에서 지원하는 불용어 사용

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text = ["Family is not an important thing. It's everything."]
# text = ["아버지가방에 들어가신다아아아아아. 안녕"]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)

print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# 2. DTM (Dcoument-Term Matrix)

다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것  
다수의 문서에 대해서 bow를 하나의 행렬로 표현하고 부르는 용어

- 문서 1 : I like dog
- 문서 2 : I like cat
- 문서 3 : I like cat I like cat

In [ ]:
import pandas as pd
content = [[0, 1, 1, 1], [1, 0, 1, 1], [2, 0, 2, 2]]
df = pd.DataFrame(content)
df.index = ['(문서1) I like dog', '(문서2) I like cat', '(문서3) I like cat I like cat']
df.columns = ['cat', 'dog', 'I', 'like']
df

,cat,dog,I,like
(문서1) I like dog,0,1,1,1
(문서2) I like cat,1,0,1,1
(문서3) I like cat I like cat,2,0,2,2


코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미합니다.

나중에 이런 것을 이용하여 단어 유사도를 이용한 추천 시스템 구현할 수 있습니다.

https://wikidocs.net/24603

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])


In [ ]:
def cos_sim(a, b):
    # print(dot(a,b))
    # print(norm(a) *  norm(b))
    return dot(a, b)/ (norm(a) * norm(b))
# 코사인 유사도는 0-1 사이의 값을 가지고 1에 가까울 수록 유사하다.

print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3)) # 문장이 중복되어 있어서 유사도가 높다.

2
2.9999999999999996
0.6666666666666667
4
5.999999999999999
0.6666666666666667
6
5.999999999999999
1.0000000000000002


## 사이킷런을 이용한 CountVectorizer를 활용한 DTM 구현

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
          'John likes to watch movies',
          'Mary likes movies too',
          'Mary also likes to watch football games'
]

vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


## 3.TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)

모든 문서에 자주 등장하는 단어는 중요도가 낮다고 판단하고, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단하는 것

In [ ]:
from math import log
import pandas as pd

docs = [
          'John likes to watch movies',
          'James likes movies too',
          'Mary also likes to watch football games'
]

vocab = list(set(w for doc in docs for w in doc.split())) # 
vocab.sort()

print('단어장의 크기 : ', len(vocab))
print(vocab)

단어장의 크기 :  11
['James', 'John', 'Mary', 'also', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [ ]:
N = len(docs)
N

3

In [ ]:
# vocab = list(set(w for doc in docs for w in doc.split())) 를 풀면 아래와 같다.

from math import log
import pandas as pd

docs = [
          'John likes to watch movies',
          'James likes movies too',
          'Mary also likes to watch football games'
]

vocab = []
for doc in docs:
    for w in doc.split():
        vocab.append(w)

vocab = list(set(vocab)) # 주의 : [set(vocab)] 과 결과가 다르다.[set(vocab)]은 {} 괄호가 더 들어감
vocab.sort()
print('단어장의 크기 : ', len(vocab))
print(vocab)

단어장의 크기 :  11
['James', 'John', 'Mary', 'also', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


특정 문서 내에서 특정 단어의 빈도인 TF(Term Frequecy) 와,
전체 문서 내에서 특정 단어의 빈도인 DF(Document Frequency)의 역수를 활용하여
어떠한 단어가 얼마나 중요한지를 나타낸 통계적 수치!

이 문서에는 자주 등장하고,
다른 문서에는 덜 등장할수록 그 수치가 크게 부여된다.

장단점
장점 : 일반적인 Bag of words를 통한 단순한 비교보다 더 높은 정확성을 보인다.
단점 : 단어의 빈도로 판단할 뿐 맥락적 유사도는 반영하지 못한다.
-> Word2Vec Embedding, ConceptNet(Knowledge graph)로 개선

출처 : http://openuiz.blogspot.com/2018/11/tf-idf.html

1. tf(t, d) : 특정 문서 d에서의 특정 단어 t의 등장 횟수
2. df(t) : 특정 단어 t가 등장한 문서의 수
3. idf(t, d) : df(t)에 반비례하는 수

$$ idf(d, t) = log\frac{n}{1+df(t)}$$

In [ ]:
def tf(t, d): # 특정 문서 d에서의 특정 단어 t의 등장 횟수
    return d.count(t)

def idf(t): #특정 단어 t가 등장한 문서의 수
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))+1 # 0으로 되는 것을 방지하기 위해서 log를 사용

def tfidf(t, d): # tf * idf 값을 계산
    return tf(t, d) * idf(t)

In [ ]:
result = []
for i in range(N):
    result.append([])
    d = docs[i] # 문서
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tf(t, d))

In [ ]:
tf_ = pd.DataFrame(result, columns=vocab)
tf_

,James,John,Mary,also,football,games,likes,movies,to,too,watch
0,0,1,0,0,0,0,1,1,1,0,1
1,1,0,0,0,0,0,1,1,1,1,0
2,0,0,1,1,1,1,1,0,1,0,1


In [ ]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.405465
also,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.000000
to,0.712318
too,1.405465


In [ ]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t, d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,also,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,1.0,0.712318,0.000000,1.0
1,1.405465,0.000000,0.000000,0.000000,0.000000,0.000000,0.712318,1.0,0.712318,1.405465,0.0
2,0.000000,0.000000,1.405465,1.405465,1.405465,1.405465,0.712318,0.0,0.712318,0.000000,1.0


## scikit-learn을 활용한 TF-IDF구현

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus = [
          "you know I want you love",
          "I like you",
          "what should I do"
]

vector = CountVectorizer()

In [ ]:
print(vector.fit_transform(corpus).toarray())

[[0 1 0 1 0 1 0 2]
 [0 0 1 0 0 0 0 1]
 [1 0 0 0 1 0 1 0]]


In [ ]:
print(vector.vocabulary_)

{'you': 7, 'know': 1, 'want': 5, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())

[[0.         0.43381609 0.         0.43381609 0.         0.43381609
  0.         0.65985664]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.        ]]


In [ ]:
print(tfidfv.vocabulary_)

{'you': 7, 'know': 1, 'want': 5, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
